# Preprocessing part in which we obtain our desired sparse matrixes. Those matrixes are input for our model.

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.2


### In order to get the access to the already processed data and formed into appropriate tabels

### Please, download this data to your drive:
https://drive.google.com/file/d/1VyYH7hd7SJAMc89b8ZmJjxYxuFkmfRqy/view?usp=sharing

### Also, it is import to have data from the previous step 'rfmv.csv'

For example, download it here https://drive.google.com/file/d/1cwtTbZw6wrancDkB6raYHodrhiXUg3Ff/view?usp=sharing

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -U pyarrow > /dev/null

In [5]:
!pip install "dask[dataframe]" --upgrade > /dev/null

In [9]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn import preprocessing
from collections import Counter
import scipy.sparse as sparse
import os


### To get final parquete tabel

In [ ]:
custdf = pd.read_csv('c.csv')
del custdf['Unnamed: 0']
skudf = pd.read_csv('m.csv')
del skudf['Unnamed: 0']
transactions = pd.read_parquet("t.parquet")
plants = pd.read_csv('p.csv')
inter = pd.merge(transactions, plants, on = 'plant', how='left')

In [ ]:
inter.to_parquet("masterdf.parquet")

In [ ]:
masterdf = pd.read_parquet("masterdf.parquet")
active_cust = pd.read_csv("rfmv.csv")
masterdf = masterdf.rename(columns={"city": "plant_city"})
active_cust = active_cust.rename(columns={"Customer ID": "client_id"})

In [ ]:
cleaned_df = masterdf.merge(active_cust[['client_id','Clusters']], how='left', on='client_id')

## removed any null entries on clusters (if any)
cleaned_df = cleaned_df[cleaned_df['Clusters'].notnull()]

## append customer table
cleaned_df = cleaned_df.merge(custdf[['client_id', 'city']], how='left', 
                on='client_id')

### merge sku feature--product category--into master data
cleaned_df = cleaned_df.merge(skudf[['material', 'hier_level_1', 'hier_level_2', 'hier_level_3',
       'hier_level_4', 'vendor', 'is_private_label']], how='left', 
                on='material')

In [ ]:
del active_cust, masterdf
cleaned_df.to_parquet("cleaned_df.parquet")

Кристина, [27.09.20 08:16]
cleaned_df = pd.read_parquet("cleaned_df.parquet")
# Getting the required columns
# Get rid of unnecessary info, keep only essential columns
    
final_cleaned_df = cleaned_df

final_cleaned_df = final_cleaned_df[['material', 'hier_level_1', 'hier_level_4', 'vendor', 
                                     'is_private_label','sales_count', 'client_id','Clusters',]]

# Group together to get total quantity for each customer, 
# retaining unique customer list, eliminating duplicates and redundancy

del  final_cleaned_df,cleaned_df
cust_grouped.to_parquet("cust_grouped.parquet")
cust_grouped = final_cleaned_df.groupby(['client_id',
                                         'hier_level_1',
                                         'hier_level_4',
                                         'vendor',
                                         'is_private_label',
                                         'material',
                                        'Clusters']).sum().reset_index()

In [ ]:
# In case if you have already prepared
cust_grouped = pd.read_parquet("./drive/My Drive/cust_grouped.parquet")
cust_grouped['sales_count'] = cust_grouped['sales_count'].astype(int)

In [ ]:
idxs = cust_grouped.client_id[cust_grouped.client_id<=99].index

In [ ]:
cust_grouped_cutted = cust_grouped.iloc[idxs]
print(len(cust_grouped_cutted))

18672


In [ ]:
cust_grouped_cutted

,client_id,hier_level_1,hier_level_4,vendor,is_private_label,material,Clusters,sales_count
0,0,0.0,0.0,985.0,0.0,68306,2,0
1,0,0.0,0.0,3954.0,1.0,20898,2,1
2,0,0.0,1.0,410.0,1.0,19344,2,2
3,0,0.0,1.0,3825.0,0.0,36585,2,2
4,0,0.0,28.0,1291.0,0.0,43227,2,16
...,...,...,...,...,...,...,...,...
18667,99,1.0,1681.0,3145.0,0.0,68625,0,3
18668,99,1.0,1841.0,710.0,0.0,74973,0,1
18669,99,1.0,1841.0,2230.0,1.0,45666,0,3
18670,99,1.0,1841.0,2230.0,1.0,88210,0,1


### Build the customers matrix

In [ ]:
## There are two features that is used to build the user matrix here
## First is the customer's unique categories they buy 
## Second is the clusters they belong to

# Getting the unique categories they buy
# grouped them by the quantity per category
cust_qty = cust_grouped_cutted.groupby(['client_id', 'material'])['sales_count'].sum().unstack().fillna(0)

# Scaled value of the categories
# Scale value of the interaction matrix using sklearn.preprocessing.MinMaxScaler 
minmaxscaler = preprocessing.MinMaxScaler()
cust_qty_scaled = minmaxscaler.fit_transform(cust_qty)
cust_qty_scaled = pd.DataFrame(cust_qty_scaled)
cust_qty_scaled.index = cust_qty.index
cust_qty_scaled.columns = cust_qty.columns

## Include the Cluster features of the customers
## this is the additional feature that we want to add into the user feature
## the cluster serves as customer's "characteristics"
### we use groupby/nunique aggregation to get a one-hot encoding of each customer's clusters

cust_clus = cust_grouped.groupby(['client_id', 'Clusters'])['Clusters'].nunique().unstack().fillna(0)

# Merge the two features to create the user feature matrix
customer_features = pd.merge(cust_qty_scaled, cust_clus, left_index=True, right_index=True, how='inner')
#customer_features.columns()

In [ ]:
customer_features

,3_x,13,16,21,24,28,35,39,51,53,59,77,84,93,94,96,100,109,116,126,138,159,161,164,173,179,180,184,186,209,214,216,231,232,240,242,250,282,310,311,...,105405,105406,105419,105433,105435,105436,105443,105449,105458,105463,105472,105478,105490,105496,105503,105512,105517,105524,105531,105548,105552,105554,105570,105589,105617,105618,105621,105623,105632,105638,105648,105662,105664,105665,105678,105688,0,1,2,3_y
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
customer_features = customer_features.rename(columns={'3_x': 'Cluster 0', 
                                                      '0': 'Cluster 1', 
                                                      '1': 'Cluster 2', 
                                                      	'2': 'Cluster 3',
                                                      	'3_y':'Cluster 4'})

### Build the interaction matrix

In [ ]:
# Getting the Customer's transaction for every material interms of sales quantity
interactions = cust_grouped_cutted.groupby(['client_id', 'material'])['sales_count'].sum().unstack().fillna(0)

# Scale value of the interaction matrix using sklearn.preprocessing.MinMaxScaler 
minmaxscaler = preprocessing.MinMaxScaler()
interactions_scaled = minmaxscaler.fit_transform(interactions)
interactions_scaled = pd.DataFrame(interactions_scaled)

## The scaled matrix lost the index (customer) and column (item) information
## we re-append the customer ID and the material into the DataFrame's index and column respectively
interactions_scaled.index = interactions.index
interactions_scaled.columns = interactions.columns

### Build the item info matrix

In [ ]:
## We use the different categories of the items as the feature to build the item feature matrix here

item_category = pd.DataFrame(cust_grouped_cutted.groupby(['material', 'hier_level_1', 'hier_level_4', 'vendor'])['sales_count'].sum().unstack().fillna(0).reset_index().set_index('material'))
# item_cat = item_cat.applymap(replaceneg0)

# Scaled value of the item categories
minmaxscaler = preprocessing.MinMaxScaler()
item_category_scaled = minmaxscaler.fit_transform(item_category)
item_category_scaled = pd.DataFrame(item_category_scaled)
item_category_scaled.index = item_category.index
item_category_scaled.columns = item_category.columns

In [ ]:
interaction_f = sparse.coo_matrix(interactions_scaled)
user_f  = sparse.coo_matrix(customer_features) 
item_f  = sparse.coo_matrix(item_category_scaled)

### Since these data are sparse, we save them in a sparse format.

In [ ]:
sparse.save_npz("drive/My Drive/npz_data/interaction_f.npz", interaction_f)
sparse.save_npz("drive/My Drive/npz_data/user_f.npz", user_f)
sparse.save_npz("drive/My Drive/npz_data/item_f.npz", item_f)

# Saved sparse matrixes you can find here:
https://drive.google.com/drive/folders/1CaJiTPw5B-qLjh_46G84sujRQ_TtqNQS?usp=sharing